# Demonstration of the QLSAs workflow, following the qiskit patterns framework

In [ ]:
import numpy as np
import cudaq

from cudaq_qlsa.qlsa.hhl import HHL
from cudaq_qlsa.generator import generate_problem
from cudaq_qlsa.noise_model import NoiseModeler
from cudaq_qlsa.executer import Executer
from cudaq_qlsa.post_processor import Post_Processor
from cudaq_qlsa.solver import QuantumLinearSolver
from cudaq_qlsa.refiner import Refiner

np.set_printoptions(edgeitems=30, linewidth=100000)
print('Cudaq version:', cudaq.__version__)
print('Running on backend', cudaq.get_target())

# Step 0: **Define** the (classical) problem

In [ ]:
prob = generate_problem(n=8, cond_number=5.0, sparsity=0.5, seed=0)
A, b = prob["A"], prob["b"]

A  = A / np.linalg.norm(b)
b = b / np.linalg.norm(b)

print(f"A: {A}")
print()
print(f"b: {b}")

## Step 1: **Map** problem to quantum circuits and operators

In [ ]:
hhl = HHL(
    readout = 'measure_x',
    # swap_test_vector = np.ones(len(b)) / np.linalg.norm(np.ones(len(b))),
    num_qpe_qubits = 3,
    t0 = 2 * np.pi)

In [ ]:
kernel, args = hhl.build_circuit(A, b)

In [ ]:
hhl.draw(A, b)

## Step 2: **Optimize** for target hardware

Cudaq does not have dedicated transpile. Also, backends only vary between CPU and GPU. 

In [ ]:
'''
NOT WORKING IN Cudaq 0.12.0 or older 
'''
# resources = cudaq.estimate_resources(kernel, args)
# print(resources)

## Step 3: **Execute** on target hardware

In [ ]:
noisemodeler = NoiseModeler()
noise_config = [0.5, 0.5, 'depolarization', 'depolarization']
noise_model  = noisemodeler.noise_modeler(noise_config)

In [ ]:
executer = Executer()

result = executer.run(
    kernel = kernel,
    args = args,
    backend = 'nvidia', 
    shots = 1024,
    noise_model = noise_model
    )

## Step 4: **Process** result to obtain classical solution

In [ ]:
processor = Post_Processor()
solution = processor.process(result, A, b)
solution

## Wrapping steps 1-4 together in a solver:


In [ ]:
hhl_solver = QuantumLinearSolver(
    qlsa = hhl,
    backend = 'nvidia',
    shots = 1024,
    noisemodel = noise_model,
    executer = executer,
    post_processor = processor)

hhl_solution = hhl_solver.solve(A, b)
print(f"Solution: {hhl_solution}")

## Integrate **Iterative Refinement** to improve accuracy

In [ ]:
refiner = Refiner(A = A, b = b, solver = hhl_solver)
refined_solution = refiner.refine(precision = 1e-10, max_iter = 100, plot=True)

In [ ]:
refined_solution['refined_x']

In [ ]:
np.linalg.solve(A,b)